# DENSER
My implementation of DENSER, a method for architecture selection in neural networks. The paper can be found [here](https://arxiv.org/abs/2004.11002).


In [113]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum
from scripts import utils

%load_ext autoreload
%autoreload 2

Sketch of grammar for denser:
[[], [], [], [], []] 

In [140]:
class DSGE_types(Enum):
    POOLING = 1
    CONV = 2
    ACTIVATION = 3

class DSGE_pooling(Enum):
    MAX = 1
    AVG = 2

class DSGE_activation(Enum):
    RELU = 1
    SIGMOID = 2
    TANH = 3

class DSGE_genes:
    "DSGE_encoding class. The DSGE_encoding is composed of a list of genes."
    def __init__(self, type=None, c_in=None, c_out=None, param=None):
        if type is None:
            self.random_init()
        else:
            self.init_form_encoding(type, param)
        
        if self.type == DSGE_types.CONV:
            if(c_in is not None):
                self.c_out = utils.compute_output_conv2d(c_in, self.param['kernel_size'], self.param['stride'], self.param['padding'], 1)
                self.c_in = c_in
            if(c_out is not None):
                self.c_in = utils.compute_input_conv2d(c_out, self.param['kernel_size'], self.param['stride'], self.param['padding'], 1)
                self.c_out = c_out
        else:
            if(c_in is not None):
                self.c_out = c_in
                self.c_in = c_in
            if(c_out is not None):
                self.c_in = c_out
                self.c_out = c_out
        
        self.channels = {'in': self.c_in, 'out': self.c_out}
        
    def random_init(self):
        self.type = DSGE_types(np.random.randint(1, 4))
        print(self.type)
        if self.type == DSGE_types.POOLING:
            self.param = DSGE_pooling(np.random.randint(1, 3))
        elif self.type == DSGE_types.CONV:
            self.param = {'kernel_size': np.random.randint(1, 4), 'stride': np.random.randint(1, 4), 'padding': np.random.randint(1, 4)}
        elif self.type == DSGE_types.ACTIVATION:
            self.param = DSGE_activation(np.random.randint(1, 4))
    def init_form_encoding(self, type, param=None):
        self.type = type
        if param is None:
            if self.type == DSGE_types.POOLING:
               self.param = DSGE_pooling(np.random.randint(1, 3))
            elif self.type == DSGE_types.CONV:
               self.param = {'kernel_size': np.random.randint(1, 4), 'stride': np.random.randint(1, 4), 'padding': np.random.randint(1, 4)}
            elif self.type == DSGE_types.ACTIVATION:
               self.param = DSGE_activation(np.random.randint(1, 4))
        
    def get(self):
        return self.type, self.param, self.channels



In [171]:
class GA_types(Enum):
    FEATURES = 1
    CLASSIFICATION = 2

class GA_genes:
    "GA_encoding class. The GA_encoding is composed of a list of genes."
    def __init__(self, GA_type, dsge_len=4, c_in=None, c_out=None):
        self.GA_type = GA_type
        self.dsge_len = dsge_len
        self.dsge_encoding = []
        

        if( c_in is not None):
            if self.GA_type == GA_types.FEATURES:
                self.dsge_encoding.append(DSGE_genes(DSGE_types.CONV, c_in=c_in))
                self.dsge_encoding.append(DSGE_genes(DSGE_types.ACTIVATION, c_in=self.dsge_encoding[-1].channels['out']))
                self.dsge_encoding.append(DSGE_genes(DSGE_types.POOLING, c_in=self.dsge_encoding[-1].channels['out'] ) )
                c_out = self.dsge_encoding[-1].channels['out']
        elif( c_out is not None):
            if self.GA_type == GA_types.CLASSIFICATION:
                self.dsge_encoding.append(DSGE_genes(DSGE_types.CONV, c_out=c_out))
                self.dsge_encoding.append(DSGE_genes(DSGE_types.ACTIVATION, c_out=c_out))
                self.dsge_encoding.append(DSGE_genes(DSGE_types.POOLING, c_out=c_out ) )
                c_in = self.dsge_encoding[-1].channels['in']
        self.param = {'input_channels': c_in, 'output_channels': c_out}
    def get(self):
        return self.GA_type, self.dsge_encoding, self.param
    def print(self):
        print( self.GA_type)
        for i in range(len(self.dsge_encoding)):
            print( self.dsge_encoding[i].get())
        print("param: ", self.param)
            

In [168]:
class Net_encoding:
    def __init__(self, len, num_features, num_classification, c_in, c_out):
        self.len = len
        self.num_features = num_features
        self.num_classification = num_classification
        self.GA_encoding = []
        tmp1 = c_in
        for i in range(self.num_features):
            self.GA_encoding.append(GA_genes(GA_types.FEATURES, c_in=tmp1))
            tmp1 = self.GA_encoding[-1].param['output_channels']
        for i in range(self.num_classification):
            self.GA_encoding.append(GA_genes(GA_types.CLASSIFICATION))
        
        self.param = {'input_channels': c_in, 'output_channels': c_out}
    def get(self):
        return self.GA_encoding
    def print(self):
        for i in range(self.len):
            print( self.GA_encoding[i].print())

In [183]:
Net = Net_encoding(3, 2, 5, 3, 10)
Net.print()

GA_types.FEATURES
(<DSGE_types.CONV: 2>, {'kernel_size': 2, 'stride': 1, 'padding': 1}, {'in': 3, 'out': 4})
(<DSGE_types.ACTIVATION: 3>, <DSGE_activation.RELU: 1>, {'in': 4, 'out': 4})
(<DSGE_types.POOLING: 1>, <DSGE_pooling.AVG: 2>, {'in': 4, 'out': 4})
param:  {'input_channels': 3, 'output_channels': 4}
None
GA_types.FEATURES
(<DSGE_types.CONV: 2>, {'kernel_size': 3, 'stride': 1, 'padding': 1}, {'in': 4, 'out': 4})
(<DSGE_types.ACTIVATION: 3>, <DSGE_activation.SIGMOID: 2>, {'in': 4, 'out': 4})
(<DSGE_types.POOLING: 1>, <DSGE_pooling.AVG: 2>, {'in': 4, 'out': 4})
param:  {'input_channels': 4, 'output_channels': 4}
None
GA_types.CLASSIFICATION
param:  {'input_channels': None, 'output_channels': None}
None


In [ ]:
class Net(nn.Module):
    def __init__(self, Net_encod):
        self.layers = nn.Sequential ()
        for i in range(Net_encod.len):
            for j in range(Net_encod.GA_encoding[i].dsge_len):
                self.layers.add_module(self.make_layer(Net_encod.GA_encoding[i].dsge_encoding[j].get()))
            
    
    def make_layer(self, GA_encod):
            

In [31]:
class Block(nn.Module):
    def __init__(self, gene, in_channels, out_channels, kernel_size, stride, padding, bias=False):
        super(Block, self).__init__()
       

    def forward(self, x):
        return 



class Net(nn.Module):
    "Net class for DENSER architecture: each layer is a block of convolutions, with a variable number of convolutions per block"
    def __init__(self, GA_encoding, c_input, c_output):
        super(Net, self).__init__()
        self.GA_encoding = GA_encoding

        self.layers = nn.Sequential()
        for i, gene in enumerate(GA_encoding):
            block = Block(gene)
            self.layers.add_module('block%d' % (i + 1), block)        

        self.classifier = nn.Linear(10, c_output)
    def forward(self, x):
        x = self.layers(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


class individual:
    "Individuals class. The individuals are composed of a network and an GA_encoding."
    def __init__(self, GA_encoding):
        self.GA_encoding = GA_encoding
        self.net = Net(GA_encoding, 3, 10)



In [70]:
gene = _DSGE_genes()
gene.random_init()
print(gene.get())

(<DSGE_types.ACTIVATION: 3>, <DSGE_activation.SIGMOID: 2>)
